In [99]:
import pandas as pd
import numpy as np
import os
from os.path import join
import seaborn as sns
import matplotlib.pyplot as plt
from pigeon import annotate

In [100]:
labels_df = pd.read_csv(join("data", "cis-pd", "data_labels", "CIS-PD_Training_Data_IDs_Labels.csv"))
labels_df = labels_df.sort_values(by="tremor", ascending=False).reset_index(drop=True)

In [101]:
labels_df.head()

,measurement_id,subject_id,on_off,dyskinesia,tremor
0,4913062f-b696-40c6-a0f2-87efe1dd4810,1048,3.0,3.0,4.0
1,7b2698a6-f3a5-4345-9630-25d377d58992,1048,2.0,3.0,4.0
2,40f713fe-0ac9-4cd0-809c-0a8024955a2a,1048,3.0,3.0,4.0
3,e0441156-c4b8-467c-8f4f-3b532d594d8f,1048,4.0,3.0,4.0
4,daf11494-e6fa-4376-a78a-86c683885764,1004,4.0,4.0,4.0


In [102]:
m_id = "9a44ee78-2408-4116-bcc8-533a0a6708e6" # put a measurement ID to annotate here

m_df = pd.read_csv(join("data", "cis-pd", "training_data", f"{m_id}.csv"))
m_df.head()

,Timestamp,X,Y,Z
0,0.00,-0.134277,-0.887939,-0.093018
1,0.02,-0.143066,-0.892334,-0.082764
2,0.04,-0.156006,-0.915283,-0.096436
3,0.06,-0.151123,-0.934570,-0.119141
4,0.08,-0.151367,-0.951660,-0.124512


In [103]:
a_df = pd.read_csv(join("data", "cis-pd", "training_data_tremor_annotations", f"{m_id}.csv"))
a_df.head()

,interval_start,annotation
0,0.0,no_tremor
1,0.5,tremor
2,1.0,no_tremor
3,1.5,no_tremor
4,2.0,no_tremor


In [106]:
tremor_indices = np.nonzero(a_df["annotation"].str.startswith("tremor").values)[0]
around_threshold = 5
around_tremor_indices = np.sort(np.unique(np.concatenate([tremor_indices + x for x in range(-around_threshold, around_threshold)])))
around_tremor_indices = around_tremor_indices[(around_tremor_indices >= 0) & (around_tremor_indices < a_df.shape[0])]

In [107]:
around_tremor_df = a_df.iloc[around_tremor_indices]
around_tremor_df.head()

,interval_start,annotation
0,0.0,no_tremor
1,0.5,tremor
2,1.0,no_tremor
3,1.5,no_tremor
4,2.0,no_tremor


In [108]:
interval_size = 0.5

In [109]:
m_molten_df = m_df.melt(id_vars=["Timestamp"], value_vars=["X", "Y", "Z"], var_name="dim")
m_molten_df.head()

,Timestamp,dim,value
0,0.00,X,-0.134277
1,0.02,X,-0.143066
2,0.04,X,-0.156006
3,0.06,X,-0.151123
4,0.08,X,-0.151367


In [110]:
def display_interval(interval_start):
    interval_end = interval_start + interval_size
    extended_interval_molten_df = m_molten_df.loc[(m_molten_df["Timestamp"] >= interval_start - 2*interval_size) & (m_molten_df["Timestamp"] < interval_end + 2*interval_size)]
    
    far_left_molten_df = extended_interval_molten_df.loc[(extended_interval_molten_df["Timestamp"] >= interval_start - 2*interval_size) & (extended_interval_molten_df["Timestamp"] < interval_start - interval_size)]
    left_molten_df = extended_interval_molten_df.loc[(extended_interval_molten_df["Timestamp"] >= interval_start - interval_size) & (extended_interval_molten_df["Timestamp"] < interval_start)]
    interval_molten_df = extended_interval_molten_df.loc[(extended_interval_molten_df["Timestamp"] >= interval_start) & (extended_interval_molten_df["Timestamp"] < interval_end)]
    right_molten_df = extended_interval_molten_df.loc[(extended_interval_molten_df["Timestamp"] >= interval_end) & (extended_interval_molten_df["Timestamp"] < interval_end + interval_size)]
    far_right_molten_df = extended_interval_molten_df.loc[(extended_interval_molten_df["Timestamp"] >= interval_end + interval_size) & (extended_interval_molten_df["Timestamp"] < interval_end + 2*interval_size)]
    
    fig, axes = plt.subplots(1, 5, sharey=True, figsize=(20, 5))
    
    ax = sns.lineplot(x="Timestamp", y="value", hue="dim", ax=axes[0], data=far_left_molten_df)
    ax.set_axis_off()
    ax.get_legend().remove()
    ax = sns.lineplot(x="Timestamp", y="value", hue="dim", ax=axes[1], data=left_molten_df)
    ax.set_axis_off()
    ax.get_legend().remove()
    ax = sns.lineplot(x="Timestamp", y="value", hue="dim", ax=axes[2], data=interval_molten_df)
    ax = sns.lineplot(x="Timestamp", y="value", hue="dim", ax=axes[3], data=right_molten_df)
    ax.set_axis_off()
    ax.get_legend().remove()
    ax = sns.lineplot(x="Timestamp", y="value", hue="dim", ax=axes[4], data=far_right_molten_df)
    ax.set_axis_off()
    ax.get_legend().remove()
    plt.ylim(-1.3, 1.3)
    plt.show()

In [111]:
annotations = annotate(
  list(around_tremor_df["interval_start"].values),
  options=['tremor.wide', 'tremor.large', 'tremor.small', 'no_tremor', 'no_tremor.flat'],
  display_fn=lambda interval_start: display_interval(interval_start)
)

HTML(value='0 examples annotated, 1115 examples left')

Output()

In [84]:
annotations

[(6.0, 'no_tremor'),
 (6.5, 'no_tremor'),
 (7.0, 'no_tremor'),
 (7.5, 'tremor.large'),
 (8.0, 'tremor.large'),
 (8.5, 'tremor.large'),
 (9.0, 'tremor.large'),
 (9.5, 'no_tremor'),
 (10.0, 'no_tremor'),
 (10.5, 'no_tremor'),
 (34.5, 'no_tremor'),
 (35.0, 'no_tremor'),
 (35.5, 'no_tremor'),
 (36.0, 'tremor.large'),
 (36.5, 'tremor.small'),
 (37.0, 'tremor.small'),
 (37.5, 'tremor.large'),
 (38.0, 'tremor.large'),
 (38.5, 'no_tremor'),
 (39.0, 'tremor.large'),
 (39.5, 'tremor.large'),
 (40.0, 'no_tremor'),
 (40.5, 'no_tremor'),
 (41.0, 'tremor.large'),
 (41.5, 'no_tremor'),
 (42.0, 'no_tremor'),
 (42.5, 'no_tremor'),
 (43.0, 'no_tremor'),
 (136.0, 'no_tremor'),
 (136.5, 'no_tremor'),
 (137.0, 'no_tremor'),
 (137.5, 'no_tremor'),
 (138.0, 'no_tremor'),
 (138.5, 'tremor.large'),
 (139.0, 'tremor.small'),
 (139.5, 'no_tremor'),
 (140.0, 'no_tremor'),
 (140.5, 'no_tremor'),
 (141.0, 'no_tremor'),
 (142.0, 'no_tremor'),
 (142.5, 'no_tremor'),
 (143.0, 'no_tremor'),
 (143.5, 'no_tremor'),
 (144

In [85]:
a_dicts = [ { "interval_start": x[0], "annotation": x[1] } for x in annotations ]

In [86]:
a2_df = pd.DataFrame(data=a_dicts)
a2_df = a2_df.sort_values(by="interval_start", ascending=True)
a2_df.head()

,interval_start,annotation
0,6.0,no_tremor
1,6.5,no_tremor
2,7.0,no_tremor
3,7.5,tremor.large
4,8.0,tremor.large


In [87]:
a2_df.to_csv(join("data", "cis-pd", "training_data_tremor_annotations", f"{m_id}_review.csv"), index=False)

In [88]:
a3_df = a2_df.append(a_df, ignore_index=True)
a3_df = a3_df.drop_duplicates(subset=['interval_start'], keep='first')
a3_df = a3_df.sort_values(by="interval_start", ascending=True)
a3_df.head()

,interval_start,annotation
990,0.0,no_tremor
991,0.5,no_tremor
992,1.0,no_tremor
993,1.5,no_tremor
994,2.0,no_tremor


In [89]:
a3_df.to_csv(join("data", "cis-pd", "training_data_tremor_annotations", f"{m_id}_v2.csv"), index=False)